In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics


In [3]:
train_data = pd.read_csv('fraudTrain.csv')
test_data = pd.read_csv('fraudTest.csv')

In [4]:
train_data.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [5]:
train_data['is_fraud'].value_counts()

0    1289169
1       7506
Name: is_fraud, dtype: int64

In [6]:
# the data is unbalanced

In [7]:
#dropping those colums that are less relevant
cols_to_drop = ['Unnamed: 0','cc_num','merchant','first','last','trans_num','unix_time','street','category']
train_data.drop(columns=cols_to_drop,inplace = True)
test_data.drop(columns=cols_to_drop,inplace = True)


In [8]:
train_data['lat_dist'] = abs(round(train_data['merch_lat']-train_data['lat'],2))
train_data['long_dist'] = abs(round(train_data['merch_long']-train_data['long'],2))

test_data['lat_dist'] = abs(round(test_data['merch_lat']-test_data['lat'],2))
test_data['long_dist'] = abs(round(test_data['merch_long']-test_data['long'],2))

cols_to_drop = ['trans_date_trans_time','city','lat','long','job','dob','merch_lat','merch_long','state']
train_data.drop(columns=cols_to_drop,inplace = True)
test_data.drop(columns=cols_to_drop,inplace = True)

In [10]:
train_data.head()
# data is clean now

,amt,gender,zip,city_pop,is_fraud,lat_dist,long_dist
0,4.97,F,28654,3495,0,0.07,0.87
1,107.23,F,99160,149,0,0.27,0.02
2,220.11,M,83252,4154,0,0.97,0.11
3,45.00,M,59632,1939,0,0.80,0.45
4,41.96,M,24433,99,0,0.25,0.83


In [12]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [25]:
le.fit(train_data['gender'])

LabelEncoder()

In [34]:
train_data[(train_data['gender'] == "M")]["gender"] = 0

C:\Users\omark\AppData\Local\Temp\ipykernel_12152\1000120518.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[(train_data['gender'] == "M")]["gender"] = 0


In [35]:
train_data.gender =[ 1 if value == "M" else 0 for value in train_data.gender]
test_data.gender =[ 1 if value == "M" else 0 for value in test_data.gender]

In [36]:
train_data

,amt,gender,zip,city_pop,is_fraud,lat_dist,long_dist
0,4.97,0,28654,3495,0,0.07,0.87
1,107.23,0,99160,149,0,0.27,0.02
2,220.11,1,83252,4154,0,0.97,0.11
3,45.00,1,59632,1939,0,0.80,0.45
4,41.96,1,24433,99,0,0.25,0.83
...,...,...,...,...,...,...,...
1296670,15.56,1,84735,258,0,0.88,0.79
1296671,51.70,1,21790,100,0,0.36,0.74
1296672,105.93,1,88325,899,0,0.68,0.69
1296673,74.90,1,57756,1126,0,0.56,0.70


In [37]:
X_train = train_data.drop('is_fraud',axis=1)
X_test = test_data.drop('is_fraud',axis=1)
y_train = train_data['is_fraud']
y_test = test_data['is_fraud']

In [38]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_trian = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
# using decision trees
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state = 45)
dtc.fit(X_train,y_train)
y_pred = dtc.predict(X_test)

In [41]:
from sklearn.metrics import accuracy_score

In [42]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 1.00
